Searching for news based on given keyword!

In [ ]:
!pip install newsapi-python

In [ ]:
from newsapi import NewsApiClient

In [ ]:
newsapi = NewsApiClient(api_key='ac5de7053a6e4d6fa1a29f1071bfe1be')

In [ ]:
news_sources=newsapi.get_sources()
for source in news_sources['sources']:
  print(source['name'])

In [ ]:
news_sources

In [ ]:
top_headlines=newsapi.get_top_headlines(q='Ukraine Russia',language='en')   #// HERE PASS QUERY WHICH YOU WANT SEARCH FOR NEWS SUMMARY
for article in top_headlines['articles']:
  print('Title:',article['title'])
  print('description:',article['description'],'\n\n')

In [ ]:
url_link=[]
for article in top_headlines['articles']:
  url_link.append(article['url'])
print(url_link)

In [ ]:
all_articles = newsapi.get_everything(
    q='World War',
    language='en',   
)
for article in all_articles['articles']:
    print('Source : ',article['source']['name'])
    print('Title : ',article['title'])
    print('URL : ',article['url'])
    print('Description : ',article['description'],'\n\n')

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install lxml

In [ ]:
!pip install nltk

In [ ]:
import bs4 as bs
from urllib.request import Request,urlopen
import re
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/24 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
article_text = ""
#url_link=['https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/']
for url in url_link:
  request = Request(url, headers=hdr)
  scraped_data = urlopen(request)
  # scraped_data = urllib.request.urlopen(url,headers=hdr)
  article = scraped_data.read()
  parsed_article = bs.BeautifulSoup(article,'lxml')
  paragraphs = parsed_article.find_all('p')
  for p in paragraphs:
    article_text += p.text

In [ ]:
print(article_text)

In [ ]:
article_text = re.sub(r'[[0-9]*]', ' ', article_text)#[10][20]
print(article_text)
article_text = re.sub(r'/\s+/g', ' ', article_text)

In [ ]:
article_text

In [ ]:
# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
formatted_article_text = re.sub(r'/\s+/g', ' ', formatted_article_text)


Tokenisng the *text*

In [ ]:
import nltk
nltk.download('punkt')
sentence_list=nltk.sent_tokenize(article_text)

In [ ]:
#Check if a word is present in the stopwords.Keep the word:frequency information
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
word_frequencies={}
for word in nltk.word_tokenize(formatted_article_text):
  if word not in stopwords:
    if word not in word_frequencies.keys():
      word_frequencies[word]=1
    else:
      word_frequencies[word]+=1


In [ ]:
#To check word with maximum frequency
maximum_frequency=max(word_frequencies.values())
for word in word_frequencies.keys():
  word_frequencies[word]=(word_frequencies[word]/maximum_frequency)

In [ ]:
word_frequencies

In [ ]:
sentence_scores={}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [ ]:
sentence_scores

In [ ]:
import heapq
summary_sentences=heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
summary_sentences

In [ ]:
summary=' '.join(summary_sentences)
print(summary)